In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import branca
import warnings
from pathlib import Path
import flopy
from matplotlib.lines import Line2D
warnings.filterwarnings(action='ignore')

In [ ]:
cwd = os.getcwd()
base_model_dir = Path('../Base_Model_for_Scenarios').resolve()
obs_locs_dir =  Path('../Locations/Obs_locs').resolve()
refined_truth_dir = Path('../Refined_Truth_Model/Modflow_Input_Files').resolve()
base_modelgrid_dir =  Path('../Locations/Base_Model_Grid').resolve()
print(cwd)
print(base_model_dir)
print(obs_locs_dir)
print(refined_truth_dir)
print(base_modelgrid_dir)

In [ ]:
# Get truth observed locations and heads
obs_locsR25 =  pd.read_csv(obs_locs_dir / 'R25.csv')
obs_locsR100 = pd.read_csv(obs_locs_dir / 'R100.csv')
obs_locsC25 = pd.read_csv(obs_locs_dir / 'C25.csv')
obs_locsC100 = pd.read_csv(obs_locs_dir / 'C100.csv')

In [ ]:
obs_locsR25 = gpd.GeoDataFrame(obs_locsR25, geometry=gpd.points_from_xy(obs_locsR25.x, obs_locsR25.y))
obs_locsR100 = gpd.GeoDataFrame(obs_locsR100, geometry=gpd.points_from_xy(obs_locsR100.x, obs_locsR100.y))
obs_locsC25 = gpd.GeoDataFrame(obs_locsC25, geometry=gpd.points_from_xy(obs_locsC25.x, obs_locsC25.y))
obs_locsC100 = gpd.GeoDataFrame(obs_locsC100, geometry=gpd.points_from_xy(obs_locsC100.x, obs_locsC100.y))

In [ ]:
# Get model_root_name from listing file
for file in os.listdir(base_model_dir):
    if file.endswith(".dis"):
        disfile = file
gwfmodel_root_name = os.path.splitext(disfile)[0]
print(gwfmodel_root_name)

# Load model
sim = flopy.mf6.MFSimulation.load(f'{gwfmodel_root_name}', sim_ws=base_model_dir, exe_name='mf6.exe')
model = sim.get_model(f'{gwfmodel_root_name}')

In [ ]:
# Get true capture
true_cap = gpd.read_file(os.path.join(refined_truth_dir,'true_cap.shp'), engine='pyogrio')
# dissolve to just a few polygons
true_cap = true_cap.dissolve().explode()

In [ ]:
# Get Riv polyline and GHB Line
rivline = gpd.read_file(os.path.join(base_modelgrid_dir,'river_polyline.shp'), engine='pyogrio')
ghbline = gpd.read_file(os.path.join(base_modelgrid_dir,'ghb_line.shp'), engine='pyogrio')

In [ ]:
# Get well loc dots
well_locs = gpd.read_file(os.path.join(base_modelgrid_dir, 'Wells_locs.shp'), engine='pyogrio')

In [ ]:
# Get truth heads
truth_heads_contours = gpd.read_file(os.path.join(base_modelgrid_dir, 'heads1.shp'), engine='pyogrio')

In [ ]:
# legend items
obswell_dot = Line2D([],[], color='navy', marker='o', markeredgecolor='none', linestyle='none', markersize=6, label='Observation Location')
pumpingwell_dot = Line2D([],[], color='aqua', marker='o', markeredgecolor='none', linestyle='none', markersize=6, label='Pumping Location')
sim_contours_line =  Line2D([0],[0], color='gray', marker='none', linestyle='dotted', linewidth=2, label='Truth Potentiometric Surface')
riv_line =  Line2D([0],[0], color='green', marker='none', linewidth=3, label='River')
ghb_line =  Line2D([0],[0], color='m', marker='none', linewidth=3, label='General Head Boundary')
true_cap_line =  Line2D([0],[0], color='red', marker='none', linewidth=3, label='Truth Capture')

LegendElement = [obswell_dot, pumpingwell_dot, sim_contours_line, riv_line, ghb_line, true_cap_line]

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.rcParams['font.family'] = 'Calibri'
ax1 = plt.subplot2grid((7,15), (0,0), rowspan=5, colspan=3) # (subplot grid, num rows, num cols), (start row, start col)
ax2 = plt.subplot2grid((7,15), (0,3), rowspan=5, colspan=3)
ax3 = plt.subplot2grid((7,15), (0,6), rowspan=5, colspan=3)
ax4 = plt.subplot2grid((7,15), (0,9), rowspan=5, colspan=3)
axleg = plt.subplot2grid((7,15),(5,0), rowspan=1, colspan=12)

# ax1
ax1.set_xticks([])
ax1.set_yticks([])
mapview = flopy.plot.PlotMapView(model, ax=ax1)
quadmesh = mapview.plot_ibound()
#wel = mapview.plot_bc("WEL", color='red')
#rivs = mapview.plot_bc("RIV", plotAll=True, color="green")
#ghbs = mapview.plot_bc("GHB", plotAll=True, color="purple")
rivline.plot(ax=ax1, color='green')
ghbline.plot(ax=ax1, color='m')
obs_locsR25.plot(ax=ax1, markersize=5, color='navy', zorder=10)
well_locs.plot(ax=ax1, markersize=5, color='aqua', zorder=10)
true_cap.boundary.plot(ax=ax1, edgecolor='red', zorder=1)
truth_heads_contours.plot(ax=ax1, colors='gray', linestyles='dotted', linewidths=1)

# ax2
ax2.set_xticks([])
ax2.set_yticks([])
mapview = flopy.plot.PlotMapView(model, ax=ax2)
quadmesh = mapview.plot_ibound()
wel = mapview.plot_bc("WEL", color='red')
rivline.plot(ax=ax2, color='green')
ghbline.plot(ax=ax2, color='m')
obs_locsR100.plot(ax=ax2, markersize=5, color='navy', zorder=10)
well_locs.plot(ax=ax2, markersize=5, color='aqua', zorder=10)
true_cap.boundary.plot(ax=ax2, edgecolor='red', zorder=1)
truth_heads_contours.plot(ax=ax2, colors='gray', linestyles='dotted', linewidths=1)

# ax3
ax3.set_xticks([])
ax3.set_yticks([])
mapview = flopy.plot.PlotMapView(model, ax=ax3)
quadmesh = mapview.plot_ibound()
wel = mapview.plot_bc("WEL", color='red')
rivline.plot(ax=ax3, color='green')
ghbline.plot(ax=ax3, color='m')
obs_locsC25.plot(ax=ax3, markersize=5, color='navy', zorder=10)
well_locs.plot(ax=ax3, markersize=5, color='aqua', zorder=10)
true_cap.boundary.plot(ax=ax3, edgecolor='red', zorder=1)
truth_heads_contours.plot(ax=ax3, colors='gray', linestyles='dotted', linewidths=1)

# ax4
ax4.set_xticks([])
ax4.set_yticks([])
mapview = flopy.plot.PlotMapView(model, ax=ax4)
quadmesh = mapview.plot_ibound()
wel = mapview.plot_bc("WEL", color='red')
rivline.plot(ax=ax4, color='green')
ghbline.plot(ax=ax4, color='m')
obs_locsC100.plot(ax=ax4, markersize=5, color='navy', zorder=10)
well_locs.plot(ax=ax4, markersize=5, color='aqua', zorder=10)
true_cap.boundary.plot(ax=ax4, edgecolor='red', zorder=1)
truth_heads_contours.plot(ax=ax4, colors='gray', linestyles='dotted', linewidths=1)

# axleg
axleg.axis('off')
axleg.legend(handles=LegendElement, frameon=False, ncol=3, prop= {'size':8, 'family':'calibri'}) 

plt.gcf().text(0.15, 0.885, 'Random 25', fontsize=12)
plt.gcf().text(0.3, 0.885, 'Random 100', fontsize=12)
plt.gcf().text(0.46, 0.885, 'Clustered 25', fontsize=12)
plt.gcf().text(0.61, 0.885, 'Clustered 100', fontsize=12)

fig.savefig('Figure1.png', bbox_inches='tight', dpi=300)